In [1]:
from utils.functions import *
from datasets import Dataset

%load_ext autoreload
%autoreload 2

### Step 1: Loading the model and data

In [2]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
goal_function = UntargetedClassification(model_wrapper)
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


Separating train and test data

In [3]:
train_data, test_data = get_data("ag_news")

Using custom data configuration default
Reusing dataset ag_news (/home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-2c14abbb593f9a7d.arrow
Loading cached shuffled indices for dataset at /home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-03c5e7de52806366.arrow


In [42]:
df_bae = pd.read_csv(r"data/bae.csv")
df_bae["label"] = df_bae.apply(
    lambda x: 1
    if x.result_type == "Successful"
    else -1
    if x.result_type == "Skipped"
    else 0,
    axis=1,
)
#df_bae["label"] = df_bae.label.apply(
#    lambda x: -1 if (x + np.random.uniform()) < 0.8 else x)

test_data = df_bae[df_bae.label > -1]
test_data.rename(columns={"original_text": "text"}, inplace=True)
test_data = test_data[['text','label']]
test_data = Dataset.from_pandas(test_data)

/opt/mamba/envs/env_nlp/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Charging last layer outputs

In [43]:
#dataset = load_dataset("ag_news").shuffle(seed=0)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=False)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [48]:
#last_layer, pred, softmax = run_model(test_data_loader, output_hs=True)
#torch.save(last_layer,'last_layer')
#torch.save(softmax,'softmax')
last_layer, softmax = torch.load('last_layer'),torch.load('softmax')

Results = pd.DataFrame()
for method in ['Softmax','Kullback-Leibler','Mahalanobis']:
    Results[method] = get_score(last_layer,pred,softmax,method=method)

Mahalanobis

In [56]:
#last_layer_train = run_model(train_data_loader, output_hs=True)[0]
last_layer_train = torch.load('last_layer_train')
score_maha = -get_mahalanobis_score(last_layer, last_layer_train, cov_estimator="LW")
labels = test_data["label"]
threshold_maha, results = set_threshold(score_maha, labels, fpr_threshold=0.2)
pred_maha = score_maha > threshold_maha
print(results)

100%|██████████| 100/100 [01:52<00:00,  1.13s/it]

{'tpr_at_threshold': 0.7040618955512572, 'fpr_at_threshold': 0.1999354526383734, 'AUC': 0.8385887568359184}


Softmax

In [49]:
score_softmax = -softmax.amax(1)
threshold_softmax, results = set_threshold(score_softmax, labels, fpr_threshold=0.2)
pred_softmax = score_softmax > threshold_softmax
print(threshold_softmax, results)

-0.99906915 {'tpr_at_threshold': 0.8007736943907157, 'fpr_at_threshold': 0.19864450540584153, 'AUC': 0.8742019364832736}


Thabut Score

In [50]:
softmax_sorted = softmax.sort()[0]
thabut_score = softmax_sorted[:, -2] - softmax_sorted[:, -1]
threshold_thabut, results = set_threshold(thabut_score, labels, fpr_threshold=0.2)
pred_thabut = thabut_score > threshold_thabut
print(threshold_thabut, results)

-0.998335 {'tpr_at_threshold': 0.8017408123791102, 'fpr_at_threshold': 0.1979990317895756, 'AUC': 0.8743579207384619}


Kullback_Leibler

In [51]:
score_KL = -rel_entr(softmax.detach(), [0.25] * 4).sum(1)
threshold_KL, results = set_threshold(score_KL, labels, fpr_threshold=0.2)
pred_KL = score_KL > threshold_KL
print(threshold_KL, results)

-1.3782212889203782 {'tpr_at_threshold': 0.7998065764023211, 'fpr_at_threshold': 0.19622397934484428, 'AUC': 0.8742915162356277}


Wasserstein Distance

In [52]:
score_Was = torch.Tensor(
    [-wasserstein_distance(soft, [0.25] * 4) for soft in softmax.detach()]
)
threshold_Was, results = set_threshold(score_Was, labels, fpr_threshold=0.2)
pred_Was = score_Was > threshold_Was
print(threshold_Was, results)

-0.37453458 {'tpr_at_threshold': 0.8007736943907157, 'fpr_at_threshold': 0.19864450540584153, 'AUC': 0.8741773566731765}


#### ANNEXE

Saving last layer embeddings

In [8]:
#train_data_df = pd.DataFrame(train_data)
Split_indexes = np.linspace(0,len(train_data),13).astype(int)
train_data_loaders = []
window = Split_indexes[1]-Split_indexes[0]
for split in Split_indexes[:-1]:
    train_data_loaders.append(torch.utils.data.DataLoader(\
                    Dataset.from_dict(train_data[split:split+window]), batch_size=32, shuffle=True))
for idx,data_loader in enumerate(train_data_loaders):
    last_layer_train = run_model(data_loader, output_hs=True)[0]
    torch.save(last_layer_train,f'last_layer_train {idx+1}')
last_layers = [torch.load(f'last_layer_train {idx+1}') for idx in range(12)]
torch.save(torch.cat(last_layers),'last_layer_train')